# Image Preprocessing and Binary Classification with Keras

## Objective

In this week's exercise, you will:
1. Learn how to do image preprocessing in Keras
2. Build a multilayer neural network for binary classification
3. Train the model on a real-world dataset of cats and dogs
4. Monitor performance using a validation dataset

---

## Step 1: Import Libraries

Let's start by importing the necessary libraries.

In [2]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Dense, Dropout, Flatten, Input, MaxPooling2D, Rescaling

---

## Step 2: Load and Preprocess the Data

Use `tfds.load()` to load the "cats_vs_dogs" dataset.

Find a way to split the dataset into a training and a validation set.

Also research how to apply necessary preprocessing to the data and do so (some of the preprocessing can also later be done using layers of the model).

*Note*: You can also get the dataset from other sources. However, there are some known issues with corrupted images, which then need to be addressed.

In [3]:
IMG_WIDTH = 150
IMG_HEIGHT = 150

def preprocess(image, label):
    image = tf.image.resize(image, (IMG_WIDTH, IMG_HEIGHT))
    image = tf.cast(image, tf.float32) / 255.0  # Normalize pixel values
    return image, label


# Load the dataset and split it
dataset, info = tfds.load('cats_vs_dogs', with_info=True, as_supervised=True)

# For 'cats_vs_dogs', which only has a 'train' split, we'll take a subset for validation manually.
# Let's get the number of examples and manually split.
num_examples = info.splits['train'].num_examples
train_size = int(0.8 * num_examples)
val_size = num_examples - train_size

full_dataset = dataset['train'].map(preprocess)
train_dataset = full_dataset.take(train_size).batch(32).prefetch(tf.data.AUTOTUNE)
validation_dataset = full_dataset.skip(train_size).take(val_size).batch(32).prefetch(tf.data.AUTOTUNE)

print(f"Number of training examples: {tf.data.experimental.cardinality(train_dataset).numpy() * 32}")
print(f"Number of validation examples: {tf.data.experimental.cardinality(validation_dataset).numpy() * 32}")

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/cats_vs_dogs/incomplete.SIXME6_4.0.1/cats_vs_dogs-train.tfrecord*...:   0%…

Dataset cats_vs_dogs downloaded and prepared to /root/tensorflow_datasets/cats_vs_dogs/4.0.1. Subsequent calls will reuse this data.
Number of training examples: 18624
Number of validation examples: 4672


---

## Step 3: Build a Multilayer Neural Network

Build a multilayer neural network for binary classification. Apply your knowledge from the Coursera lectures to choose an adequate model architecture.

In [4]:
model = Sequential([
    Input(shape=(IMG_WIDTH, IMG_HEIGHT, 3)),
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5), # Add dropout for regularization
    Dense(1, activation='sigmoid') # Output layer for binary classification
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 148, 148, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 74, 74, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 72, 72, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 36, 36, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 34, 34, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 17, 17, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 36992)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     4,735,104 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,828,481 (18.42 MB)

 Trainable params: 4,828,481 (18.42 MB)

 Non-trainable params: 0 (0.00 B)

---

## Step 4: Train the Model

Train the model using the training dataset you created. Monitor performance during training using the validation dataset.

In [ ]:
history = model.fit(
    train_dataset,
    epochs=10,  # You can adjust the number of epochs
    validation_data=validation_dataset
)

Epoch 1/10
189/582 ━━━━━━━━━━━━━━━━━━━━ 9:03 1s/step - accuracy: 0.5359 - loss: 0.6937

---

## Step 5: Evaluate the Model

After training, you may upload some test images to evaluate your model.

In [ ]:
from google.colab import files
from tensorflow.keras.preprocessing import image


def load_and_predict(model):
    uploaded_files = files.upload()

    for fn in uploaded_files.keys():
        path = '/content/' + fn
        img = image.load_img(path, target_size=(150, 150))

        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0) / 255.0

        classes = model.predict(x)
        result = "a dog" if classes[0] > 0.5 else "a cat"

        print(f'The model predicts that the image is of {result}')

# Call the function to upload images and get predictions
load_and_predict(model)